In [1]:
import yaml

import jax
from brax.io import html

from rl_games.torch_runner import Runner
from rl_games.envs.brax import BraxEnv

import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

In [2]:

# Training configs and path to the saved weights of the trained agent

# Ant
config_name = 'rl_games/configs/brax/ppo_ant.yaml'
trained_network = 'runs/Ant_brax_generalized/nn/Ant_brax.pth'

# Humanoid
#config_name = 'rl_games/configs/brax/ppo_humanoid.yaml'
#trained_network = 'runs/Humanoid_brax/nn/Humanoid_brax.pth'

# config_name = 'rl_games/configs/brax/sac_ant.yaml'
# trained_network = 'nn/Ant_brax_sac.pth'

# config_name = 'rl_games/configs/brax/sac_humanoid.yaml'
# trained_network = 'nn/humanoid_brax_sac.pth'

#config_name = 'rl_games/configs/brax/ppo_ur5e.yaml'
#trained_network = 'nn/Ur5e_brax.pth'

#config_name = 'rl_games/configs/brax/ppo_halfcheetah.yaml'
#trained_network = 'nn/Halfcheetah_brax.pth'

#config_name = 'rl_games/configs/brax/ppo_grasp.yaml'
#trained_network = 'nn/Grasp_brax.pth'

#config_name = 'rl_games/configs/brax/ppo_reacher.yaml'
#trained_network = 'nn/Reacher_brax.pth'

# config_name = 'rl_games/configs/brax/sac_humanoid.yaml'
# trained_network = './nn/humanoid_brax_sac.pth'

In [3]:
with open(config_name, 'r') as stream:
    config = yaml.safe_load(stream)
    runner = Runner()
    runner.load(config)
    agent = runner.create_player()
    agent.restore(trained_network)

env_config = runner.params['config']['env_config']

FileNotFoundError: [Errno 2] No such file or directory: 'rl_games/configs/brax/ppo_ant.yaml'

In [ ]:
num_actors = 1
env = BraxEnv('', num_actors, **env_config)

In [ ]:
rollout = []
obs = env.reset()
total_reward = 0
num_steps = 0

is_done = False
while not is_done:
    act = agent.get_action(obs)
    obs, reward, is_done, info = env.step(act.unsqueeze(0))
    total_reward += reward.item()
    num_steps += 1

    state = agent.env.env.pipeline_init(env.state.pipeline_state.q.squeeze(0), env.state.pipeline_state.qd.squeeze(0))
    rollout.append(state)

print('Total Reward: ', total_reward)
print('Num steps: ', num_steps)

In [ ]:
from IPython.display import HTML, clear_output

HTML(html.render(env.env.sys.replace(dt=env.env.dt), rollout))